# Merge Official with Unofficial

In [12]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from IPython.display import clear_output

import settings
from utils import vplot, vprint

In [23]:
DISTILLED_DIR = settings.DATA_CONFIG['distilled_dir']
RAW_DIR = settings.DATA_CONFIG['raw_dir']
MANUAL_DIR = settings.DATA_CONFIG['manual_dir']
FRAMEWORKS = settings.DATA_CONFIG['frameworks']
FRAMEWORKS = {v: k for k, v in FRAMEWORKS.items()}

COLUMNS = ['Source','Framework','Description','PR Num', 'Keyword', 'CVE ID','Taxonomy','CWE ID','CWE Name',
           'Root Cause', 'Fixing Pattern', 'Symptom',
           'Line Addition', 'Line Deletion', 'Line Changed', 'File Changed',
           'Created At', 'Merged At', 'Time Cost (Hours)', 
           'Comment Count', 'Review Comment Count',
           'Patch URLs']

OUTPUT_FILE = MANUAL_DIR + '/vulnerability_manual.xlsx'

##### Utils

In [15]:
def get_vuln_official():
    return MANUAL_DIR + f'vulnerability_official.xlsx'
def get_vuln_unofficial():
    return MANUAL_DIR + f'vulnerability_unofficial.xlsx'

def to_commit_url(framework,commit):
    return f'https://github.com/{FRAMEWORKS[framework]}/{framework}/commit/{commit}'
def to_pr_url(framework,pr):
    return f'https://github.com/{FRAMEWORKS[framework]}/{framework}/pull/{pr}'

## Merge

In [16]:
df = pd.DataFrame(columns = COLUMNS)
df

,Source,Framework,Description,PR Num,Keyword,CVE ID,Taxonomy,CWE ID,CWE Name,Root Cause,...,Line Addition,Line Deletion,Line Changed,File Changed,Created At,Merged At,Time Cost (Hours),Comment Count,Review Comment Count,Patch URLs


#### Official Vulnerability

In [17]:
official_df = pd.read_excel(get_vuln_official(),sheet_name='manual')

In [18]:
official_df.columns

Index(['CVE ID', 'framework', 'Description', 'Taxonomy', 'Root Cause',
       'Fixing Pattern', 'Symptom', 'CWE ID', 'CWE Name', 'commit hash',
       'commit description', 'CVSS2 Access Complexity',
       'CVSS2 Authentication Required', 'CVSS2 Availability Impact',
       'CVSS2 Confidentiality Impact', 'CVSS2 Score', 'CVSS2 Integrity Impact',
       'CVSS3attackVector', 'CVSS3attackComplexity', 'CVSS3privilegesRequired',
       'CVSS3userInteraction', 'CVSS3scope', 'CVSS3confidentialityImpact',
       'CVSS3integrityImpact', 'CVSS3availabilityImpact', 'CVSS3baseScore',
       'CVSS3baseSeverity'],
      dtype='object')

In [19]:
for index, row in official_df.iterrows():
    framework   = row['framework']
    cve_id      = row['CVE ID']
    commits     = row['commit hash']
    description = row['Description']
    taxonomy    = row['Taxonomy']
    cwe_id      = row['CWE ID']
    cwe_name    = row['CWE Name']
    root_cause  = row['Root Cause']
    fixing_pattern = row['Fixing Pattern']
    symptom     = row['Symptom']
    patch_urls  = np.nan
    if commits is not np.nan:
        commits = commits.split(', ')
        patch_urls = [ to_commit_url(framework,commit) for commit in commits]
    df = df.append({'Source':'Official','Framework':framework,'Description':description,'CVE ID':cve_id,
                    'Taxonomy':taxonomy,'CWE ID':cwe_id,'CWE Name':cwe_name,
                    'Root Cause':root_cause,'Fixing Pattern':fixing_pattern,'Symptom':symptom,
                    'Patch URLs':patch_urls}, ignore_index=True)
    clear_output(wait=True)
    print(f'{index+1}/{len(official_df)}')

465/465


#### Unofficial Vulnerability

In [21]:
unofficial_df = pd.read_excel(get_vuln_unofficial(),sheet_name='manual')
# Unified Description of False Positive 
unofficial_df.loc[unofficial_df['CWE'].str.contains('false positive', case=False).fillna(False), 'CWE'] = 'false positive'
# Unified Description of `Other` 
unofficial_df.loc[unofficial_df['CWE'].str.contains('Other', case=False).fillna(False), 'CWE'] = 'Other'
# Fill Default Root Case with `Other` 
unofficial_df.loc[unofficial_df['root_cause'].isnull(), 'root_cause'] = 'Other'
# Fill Default Taxonomy with `Other` 
unofficial_df.loc[unofficial_df['Taxonomy'].str.contains('other', case=False).fillna(False), 'Taxonomy'] = 'other'
# Drop all `false positive`
unofficial_df = unofficial_df[~(unofficial_df['CWE']=='false positive')].reset_index()

KeyError: 'CWE'

In [26]:
for index, row in unofficial_df.iterrows():
    framework   = row['framework']
    pr          = row['pr_number']
    keyword     = row['keyword']
    description = row['title']
    taxonomy    = row['taxonomy']
    cwe_id      = np.nan
    cwe_name    = np.nan
    root_cause  = row['root_cause']
    fixing_pattern = row['fixing_pattern']
    symptom     = row['symptom']
    line_addition = row['line_additions']
    line_delettion = row['line_deletions']
    created_at = row['created_at']
    merged_at = row['merged_at']
    line_changed = line_addition + line_delettion
    file_changed = row['changed_files_count']
    comment_count = row['commits_count']
    review_comment_count = row['review_comments_count']
    # URLs
    patch_urls  = np.nan
    if pr is not np.nan:
        patch_urls = [to_pr_url(framework,pr)]
    df = df.append({'Source':'Unofficial','Framework':framework,'Description':description,
                    'Taxonomy':taxonomy,'CWE ID':cwe_id,'CWE Name':cwe_name,
                    'Root Cause':root_cause,'Fixing Pattern':fixing_pattern,'Symptom':symptom,
                    'Line Addition':line_addition, 'Line Deletion':line_delettion, 'Line Changed':line_changed, 
                    'File Changed':file_changed,'Created At':created_at, 'Merged At':merged_at,
                    'Comment Count':comment_count, 'Review Comment Count':review_comment_count,
                    'Patch URLs':patch_urls, 'PR Num':pr, 'Keyword':keyword}, ignore_index=True)
    clear_output(wait=True)
    print(f'{index+1}/{len(unofficial_df)}')

3163/3163


In [27]:
## Calculate Time Cost
def calc_time_cost(df):
    df_ = df[(~df['Created At'].isnull()) & (~df['Merged At'].isnull())]
    created_at = pd.to_datetime(df_["Created At"], format="%Y-%m-%d").astype(int) / 10**9
    merged_at = pd.to_datetime(df_["Merged At"], format="%Y-%m-%d").astype(int) / 10**9
    time_cost = ((merged_at - created_at) / 3600).round(2)
    df['Time Cost (Hours)'] = time_cost
calc_time_cost(df)

In [28]:
df = df[COLUMNS]
if not os.path.isfile(OUTPUT_FILE):
    df.to_excel(OUTPUT_FILE,sheet_name='manual', index=False)